## Network Analysis Project 

#### About the dataset 
I will be working with a company's email network dataset (fictional dataset; online source) where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [2]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


### Part 1 - Salary Prediction

Using network `G`, I will be identifying the people in the network with missing values for the node attribute `ManagementSalary` and predicting whether or not these individuals are receiving a managment position salary.

In [3]:
list(G.nodes(data=True))[:5]

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan})]

In [4]:
def salary_predictions():
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier

    input_data = pd.DataFrame()
    target = nx.get_node_attributes(G, 'ManagementSalary')
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    for node in target.keys():
        row_features = pd.DataFrame([[degree_centrality[node], closeness_centrality[node],
                                      betweenness_centrality[node], target[node]]], index=[node])
        input_data = pd.concat([input_data, row_features], axis=0)
    train_data = input_data[~input_data[3].isnull()]
    test_data = input_data[input_data[3].isnull()]
    clf = GradientBoostingClassifier()
    clf.fit(train_data[[0,1,2]].values, train_data[3].values)
    preds = clf.predict_proba(test_data[[0,1,2]].values)[:,1]  
    return pd.Series(preds, index=test_data.index)

salary_predictions()

1      0.050915
65     0.986815
18     0.542332
215    0.986815
283    0.986220
         ...   
691    0.008366
788    0.008597
944    0.008366
798    0.008366
808    0.008366
Length: 252, dtype: float64

### Part 2 - New Connections Prediction

For the last part of this project, I will be predicting future connections between employees of the network. (1.0 indicates a future connection)

In [5]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


#### Gradient Boosting Classifier

Using network `G` and `future_connections`, I will be identifying the edges in `future_connections` with missing values and predicting whether or not these edges will have a future connection.

In [6]:
def new_connections_predictions():
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier

    future_connections['pref_attachment'] = [list(nx.preferential_attachment(G, [node_pair]))[0][2]
                                             for node_pair in future_connections.index]
    future_connections['comm_neighbors'] = [len(list(nx.common_neighbors(G, node_pair[0], node_pair[1]))) 
                                            for node_pair in future_connections.index]
    train_data = future_connections[~future_connections['Future Connection'].isnull()]
    test_data = future_connections[future_connections['Future Connection'].isnull()]
    clf = GradientBoostingClassifier()
    clf.fit(train_data[['pref_attachment','comm_neighbors']].values, train_data['Future Connection'].values)
    preds = clf.predict_proba(test_data[['pref_attachment','comm_neighbors']].values)[:,1]
    return pd.Series(preds, index=test_data.index)

new_connections_predictions()

(107, 348)    0.031823
(542, 751)    0.012931
(20, 426)     0.543026
(50, 989)     0.013104
(942, 986)    0.013103
                ...   
(165, 923)    0.013183
(673, 755)    0.013103
(939, 940)    0.013103
(555, 905)    0.012931
(75, 101)     0.017730
Length: 122112, dtype: float64

**Thank You for Reviewing This Project**

I appreciate you taking the time to go through my work. Please feel free to reach out if you have any questions, suggestions, or would like to discuss any aspects of this project further.

Best Regards,

Chaymae
##### *Chaymaejawhar@gmail.com*